In [1]:
import fitz #PyMuPDF
import pandas as pd
import re # Regex
import glob
import os

In [2]:
def mail_number_zipcode(f,mailpattern,phonenopattern,zipcode,clientMail,clientPhone,Businesszipcode,dic):
    doc = fitz.open(f)
    fulltext = ''
    for page in doc: # iterate the document pages
        text = page.get_text()
        fulltext = fulltext + text
    # print(fulltext)
    try:
        mail = re.search(mailpattern, fulltext)
        clientMail.append(mail.group())
        dic['Client_Mail_Adrres'] = clientMail
    except:
        clientMail.append('None')
        dic['Client_Mail_Adrres'] = clientMail
        
    try:
        phoneno = re.search(phonenopattern, fulltext)
        clientPhone.append(phoneno.group())
        dic['Client_PhoneNo'] = clientPhone
        
    except:
        clientPhone.append('None')
        dic['Client_PhoneNo'] = clientPhone
        
    try:
        zipcode = re.search('\d{5}',fulltext)
        Businesszipcode.append(zipcode.group())
        dic['Business_Zipode'] = Businesszipcode
    except:
        Businesszipcode.append('None')
        dic['Business_Zipode'] = Businesszipcode
        
    return dic


In [3]:
def Key_value(f,dic,invoice,invoicedate,duedate,subtotal,tax,totaldue,clientname):
    doc = fitz.open(f)
    for page in doc: 
        words = page.get_text("words")  # extract sorted words
    #     print(words)
        for i, word in enumerate(words):
            # information items will be found prefixed with their "key"
            text = word[4]
            if text == "Invoice#":
                invoiceno = words[i + 1][4]
                invoice.append(invoiceno)
                dic['Invoice_No.'] = invoice
              
            elif text == "Issue" and words[i+1][4] == 'date':  # the following word will be the date!
                ISsuedate = words[i + 2][4]
                invoicedate.append(ISsuedate)
                dic['Invoice_IssueDate'] = ISsuedate
                
            elif text == "Due" and words[i+1][4] == 'date:':
                DueDates = words[i + 2][4]
                duedate.append(DueDates)
                dic['Due_Date'] = duedate
            
            elif text == "Subtotal":
                subtotalvalue = words[i + 1][4]
                subtotal.append(subtotalvalue)
                dic['SubTotal'] = subtotal
                
            elif text == "Tax":
                taxper = words[i + 2][4]
                tax.append(taxper)
                dic['Tax_per'] = tax
                
            elif text == "Total" and words[i+1][4] == 'Due':
                TotalDueamount = words[i + 2][4]
                totaldue.append(TotalDueamount)
                dic['Tota_Due'] = totaldue
               
            elif text == "BILL" and words[i+1][4] == 'TO':
                clientnamestring = words[i + 2][4] +' '+ words[i + 3][4]
                clientname.append(clientnamestring)
                dic['Client_Name'] = clientname
                
    return dic

In [4]:
if __name__ == '__main__':
    
    dirpath = os.getcwd()
    # print(dirpath)
    filename = 'PDFData'
    fname = os.path.join(dirpath,filename) 
    # print(fname)
    # print(os.listdir(fname))
    mailpattern = ".*_+[a-zA-Z0-9]+@yahoo\.co|.*_+[a-zA-Z0-9]+@yahoo\.c|.*_+[a-zA-Z0-9]+@hotm|.*_+[a-zA-Z0-9]+@hotmail\.co|.*\.+[a-zA-Z0-9]+@yahoo\.com|[a-zA-Z0-9]+@yahoo\.com|.*\.+[a-zA-Z0-9]+@hotmail\.com|.*\.+[a-zA-Z0-9]+@gmail\.com"
    phonenopattern = '\d{3}-\d{3}-\d{4}'
    zipcode = '\d{5}'
    dic = {}
    clientMail = []
    clientPhone = []
    Businesszipcode = []
    invoice = []
    invoicedate = []
    duedate = []
    subtotal = []
    tax = []
    totaldue = []
    clientname = []

    for f in glob.glob(f"{fname}/*.pdf"):
    #     print(f)
        #function for finding patterns zipcode, mail and phone number
        dic = mail_number_zipcode(f,mailpattern,phonenopattern,zipcode,clientMail,clientPhone,Businesszipcode,dic)
        #function for finding invoice number, invoice date, due date, sub total, tax %, total due and client name
        dic = Key_value(f,dic,invoice,invoicedate,duedate,subtotal,tax,totaldue,clientname)
    dataf = pd.DataFrame(dic)
    columsequence = ['Invoice_No.','Business_Zipode','Client_Name','Client_Mail_Adrres','Client_PhoneNo','Invoice_IssueDate','Due_Date','SubTotal','Tax_per','Tota_Due',]
    dataf.to_csv("InvoiceData.csv",columns=columsequence)
   